## ДЗ к Урок 2. Загрузка данных и построение обучающей выборки. Анализ и предобработка датасета. Балансировка классов

##### 1. Напишите функцию, которая возвращает сумму двух вещественных аргументов a и b, а к ней декоратор, который делает так, чтобы возвращаемое значение функцией было по модулю 5.

In [1]:
def mod_5(func):
    def wrapper(*args, **kwargs):
        return_value = func(*args, **kwargs) % 5
        return return_value
    return wrapper

@mod_5
def sum_func(a, b):
    c = a + b
    return c

a = 12.345
b = 23.456
c = sum_func(a, b)
print('Суммa вещественных чисел {} и {} по модулю 5 равна равна = {}'.format(a, b, c))

Суммa вещественных чисел 12.345 и 23.456 по модулю 5 равна равна = 0.8010000000000019


##### 2. Попробуйте описать своими словами основные отличия SMOTE от ADASYN, ознакомившись с документацией к ним.

Оба метода принадлежат категории __Over-sampling__.

SMOTE создает элементы в непосредственной близости от уже существующих в меньшем наборе, а так же накладывает ограничения на выбор объектов миноритарного класса, на основе которых генерируются новые экземпляры. Полученный набор представляет собой экземпляры миноритарного класса, находящиеся на границе распределения, и именно у них вероятность оказаться некорректно классифицированными выше, чем у прочих.
Идея алгоритма SMOTE состоит в том, чтобы создать «синтетические» точки данных вдоль вектора между двумя выборками из класса меньшинства, выбранного его ближайшим соседом (т.е. ближайшим). Новые точки задаются на произвольной длине вдоль этого вектора.

ADAZYN основывается на систематическом методе, позволяющем адаптивно генерировать разные количества данных в соответствии с их распределениями, то есть использует функции плотности распределения как критерий для автоматического определения числа экземпляров, которые необходимо сгенерировать для каждого из объектов миноритарного класса, адаптивно меняя веса разных экземпляров миноритарного класса.
Подобно SMOTE, метод ADASYN также генерирует новые синтетические точки, но применяя различные веса к выборкам меньшинства, чтобы компенсировать перекос распределений. В алгоритме SMOTE одинаковое количество синтетических выборок генерируется для каждого примера данных меньшинства.